In [1]:
import requests
import pandas as pd
import numpy as np
import os
import json
import datetime

In [2]:
BETTING_SITES = ["DraftKings NJ Sportsbook", "FanDuel NJ Sportsbook", "BetRivers NJ Sportsbook (1)", "BetMGM", "PointsBet", "bet365", "Caesars NJ Sportsbook", "UnibetNJ Sportsbook", "BetRivers NY Sportsbook"]
OUTCOMES = ["Away", "Home", "Draw"]

NAMES_ZIP = zip(np.repeat(BETTING_SITES, len(OUTCOMES)), np.tile(OUTCOMES, len(BETTING_SITES)))
NAMES_CONCAT = list(map(lambda x: f"{x[0]} | {x[1]}", NAMES_ZIP))
NAMES_CONCAT.append("Away")
NAMES_CONCAT.append("Home")
NAMES_CONCAT.append("Outcome")

In [3]:
def request_info(date_Ymd):
    url = f"https://api.actionnetwork.com/web/v1/scoreboard/soccer?period=game&bookIds=15,30,76,75,123,69,68,972,71,247,79&date={date_Ymd}"

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
    }
    
    response = requests.get(url, headers=headers)
    
    return response.text
today = datetime.datetime.now().strftime("%Y%m%d")
info = json.loads(request_info(today))

In [4]:
odds_all = []
for game in info["games"]:
    teams = [game["teams"][0]["full_name"], game["teams"][1]["full_name"]]
    first_is_away = game["teams"][0]["id"] == game["away_team_id"]
    if not first_is_away:
        teams = list(reversed(teams))
    
    outcome = np.nan
    if game["status"] == "complete":
        winner_id = game["winning_team_id"]
        if winner_id == game["away_team_id"]:
            outcome = 0
        elif winner_id == None:
            outcome = 1
        else:
            outcome = 2

    odds = np.array([[odd["ml_away"], odd["ml_home"], odd["draw"]] for odd in game["odds"][2:]]).flatten()
    odds = np.append(odds, [teams[0], teams[1], outcome])
    odds_all.append(odds)


In [5]:
df = pd.DataFrame(odds_all, columns=NAMES_CONCAT)
SAVE_PATH = "odds_data.csv"

if not os.path.isfile(SAVE_PATH):
    line = ",".join(NAMES_CONCAT) + "\n"
else:
    line = ""

with open(SAVE_PATH, "a+") as file:
    file.write(line)

df.to_csv(SAVE_PATH, mode="a", index=False, header=False)